In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e1/sample_submission.csv
/kaggle/input/playground-series-s3e1/train.csv
/kaggle/input/playground-series-s3e1/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e1/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e1/test.csv')
ss = pd.read_csv('/kaggle/input/playground-series-s3e1/sample_submission.csv')

In [3]:
train_df.drop('id', axis=1, inplace=True)

In [4]:
train_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500
...,...,...,...,...,...,...,...,...,...
37132,3.3438,50.0,4.936508,1.079365,1775.0,3.022222,34.19,-118.36,1.856
37133,3.7308,26.0,5.087533,0.966019,1006.0,4.316901,37.32,-121.86,1.588
37134,4.1716,52.0,4.678862,1.101485,1156.0,1.431734,37.75,-122.44,3.387
37135,2.7143,16.0,5.710074,1.068376,584.0,2.803659,38.40,-120.98,1.592


In [5]:
target = 'MedHouseVal'
column = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population','AveOccup', 'Latitude', 'Longitude']

In [6]:
from sklearn.datasets import fetch_california_housing

external_data = fetch_california_housing()
train2 = pd.DataFrame(external_data['data'])
train2[target] = external_data['target']
train2.columns = column+[target]
train_all = pd.concat([train_df,train2],axis=0).drop_duplicates()

In [7]:
train_all

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [8]:
y = train_all.pop('MedHouseVal').to_numpy()
X = np.array(train_all)
print(X[0])

[ 2.38590000e+00  1.50000000e+01  3.82716049e+00  1.11209964e+00
  1.28000000e+03  2.48698885e+00  3.46000000e+01 -1.20120000e+02]


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=0)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

(46221, 8) (46221,)
(11556, 8) (11556,)


In [10]:
params= {
     'learning_rate':0.02,
     'lambda_l1': 1.945,
     'num_leaves': 87,
     'feature_fraction': 0.79,
     'bagging_fraction': 0.93,
     'bagging_freq': 4,
     'min_data_in_leaf': 103,
     'max_depth': 17,
     'num_iterations':10000
}

In [11]:
import lightgbm as lgbm

model = lgbm.LGBMRegressor(**params)

model.fit(x_train, y_train, eval_set = (x_valid,y_valid), early_stopping_rounds =100)

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=1.945, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.945
[LightGBM] [Warning] bagging_fraction is set=0.93, subsample=1.0 will be ignored. Current value: bagging_fraction=0.93
[LightGBM] [Warning] feature_fraction is set=0.79, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.79
[LightGBM] [Warning] min_data_in_leaf is set=103, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=103
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[1]	valid_0's l2: 1.29599
[2]	valid_0's l2: 1.26321
[3]	valid_0's l2: 1.22931
[4]	valid_0's l2: 1.19963
[5]	valid_0's l2: 1.17536
[6]	valid_0's l2: 1.14641
[7]	valid_0's l2: 1.11909
[8]	valid_0's l2: 1.09189
[9]	valid_0's l2: 1.066
[10]	valid_0's l2: 1.04212
[11]	valid_0's l2: 1.02292
[12]	valid_0's l2: 0.999509
[13]	valid_0's l2: 0.975766
[14]	valid_0's l2: 0.952157
[15]	valid_0's l2: 0.930834
[16]	valid_0's l2:

LGBMRegressor(bagging_fraction=0.93, bagging_freq=4, feature_fraction=0.79,
              lambda_l1=1.945, learning_rate=0.02, max_depth=17,
              min_data_in_leaf=103, num_iterations=10000, num_leaves=87)

In [12]:
score = model.score(x_valid, y_valid)
print(score)

0.8008913644195261


In [13]:
models = []
models.append(model)

In [14]:
test_df.drop('id', axis=1, inplace=True)
x_test = test_df.to_numpy()

In [15]:
test_preds = []

for model in models:
    preds = model.predict(x_test)
    test_preds.append(preds)

In [16]:
test_preds = np.array(test_preds).mean(0)
print(test_preds)

[0.68506931 1.0640717  4.09559871 ... 1.21570977 3.51399354 3.31478732]


In [17]:
ss['MedHouseVal'] = test_preds
ss.to_csv("submission.csv", index=False)
pd.read_csv("submission.csv").head()

,id,MedHouseVal
0,37137,0.685069
1,37138,1.064072
2,37139,4.095599
3,37140,3.360541
4,37141,2.484283
